--- Day 11: Chronal Charge ---

You watch the Elves and their sleigh fade into the distance as they head toward the North Pole.

Actually, you're the one fading. The falling sensation returns.

The low fuel warning light is illuminated on your wrist-mounted device. Tapping it once causes it to project a hologram of the situation: a 300x300 grid of fuel cells and their current power levels, some negative. You're not sure what negative power means in the context of time travel, but it can't be good.

Each fuel cell has a coordinate ranging from 1 to 300 in both the X (horizontal) and Y (vertical) direction. In X,Y notation, the top-left cell is 1,1, and the top-right cell is 300,1.

The interface lets you select any 3x3 square of fuel cells. To increase your chances of getting to your destination, you decide to choose the 3x3 square with the largest total power.

The power level in a given fuel cell can be found through the following process:

*    Find the fuel cell's rack ID, which is its X coordinate plus 10.
*    Begin with a power level of the rack ID times the Y coordinate.
*    Increase the power level by the value of the grid serial number (your puzzle input).
*    Set the power level to itself multiplied by the rack ID.
*    Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
*    Subtract 5 from the power level.

For example, to find the power level of the fuel cell at 3,5 in a grid with serial number 8:

*    The rack ID is 3 + 10 = 13.
*    The power level starts at 13 * 5 = 65.
*    Adding the serial number produces 65 + 8 = 73.
*    Multiplying by the rack ID produces 73 * 13 = 949.
*    The hundreds digit of 949 is 9.
*    Subtracting 5 produces 9 - 5 = 4.

So, the power level of this fuel cell is 4.

Here are some more example power levels:

*    Fuel cell at  122,79, grid serial number 57: power level -5.
*    Fuel cell at 217,196, grid serial number 39: power level  0.
*    Fuel cell at 101,153, grid serial number 71: power level  4.

Your goal is to find the 3x3 square which has the largest total power. The square must be entirely within the 300x300 grid. Identify this square using the X,Y coordinate of its top-left fuel cell. For example:

For grid serial number 18, the largest total 3x3 square has a top-left corner of 33,45 (with a total power of 29); these fuel cells appear in the middle of this 5x5 region:
```
-2  -4   4   4   4
-4   4   4   4  -5
 4   3   3   4  -4
 1   1   2   4  -3
-1   0   2  -5  -2
```
For grid serial number 42, the largest 3x3 square's top-left is 21,61 (with a total power of 30); they are in the middle of this region:
```
-3   4   2   2   2
-4   4   3   3   4
-5   3   3   4  -4
 4   3   3   4  -3
 3   3   3  -5  -1
```
**What is the X,Y coordinate of the top-left fuel cell of the 3x3 square with the largest total power?**

Your puzzle input is 7857.

In [1]:
import unittest
import numpy as np
from scipy.signal import convolve2d, fftconvolve

In [2]:
power_level_test_data = [[3, 5, 8, 4],
                         [122, 79, 57, -5],
                         [217, 196, 39, 0],
                         [101, 153, 71, 4]]

max_power_test_data = [[18, 33, 45, 29], [42, 21, 61, 30]]

day11_sn = 7857

In [3]:
class TestIt(unittest.TestCase):
    
    def test_power_level(self):
        for x, y, sn, pl in power_level_test_data:
            with self.subTest(x=x, y=y):
                plGrid = powerLevelGrid(sn)
                self.assertEqual(powerLevel(x, y, plGrid), pl)
                
    def test_max_power(self):
        for sn, x, y, pl in max_power_test_data:
            with self.subTest(sn=sn):
                self.assertEqual(maxPower(sn), (x, y, pl))

In [4]:
def powerLevelGrid(sn):
    x = y = range(1, 301)
    xv, yv = np.meshgrid(x, y, indexing='ij')
    # Find the fuel cell's rack ID, which is its X coordinate plus 10.
    rackID = xv + 10
    # Begin with a power level of the rack ID times the Y coordinate.
    pl = rackID * yv
    # Increase the power level by the value of the grid serial number (your puzzle input).
    pl = pl + sn
    # Set the power level to itself multiplied by the rack ID.
    pl = pl * rackID
    # Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
    pl = (pl // 100) % 10
    # Subtract 5 from the power level.
    pl = pl - 5
    return pl

def powerLevel(ix, iy, plGrid):
    return plGrid[ix - 1, iy - 1]

def maxPower(sn, size=3):
    cellGrid = maxPowerGrid(powerLevelGrid(sn), size=size)
    return maxPowerFromGrid(cellGrid)

def maxPowerFromGrid(cellGrid):
    max_element = np.max(cellGrid)
    x, y = map(lambda x: x[0], np.where(cellGrid == max_element))
    return x + 1, y + 1, max_element

def maxPowerGrid(plGrid, size, convolve=fftconvolve):
    cellGrid = convolve(np.ones((size, size), dtype=np.int), plGrid, mode='valid')
    return cellGrid.astype(np.int)

# Run part 1 tests

In [5]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestIt)
unittest.TextTestRunner(verbosity=2).run(suite)

test_max_power (__main__.TestIt) ... ok
test_power_level (__main__.TestIt) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.068s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Part 1

In [6]:
x, y, power = maxPower(day11_sn)
print(f'Part 1 answer - max power is at coordinates {x},{y} (power was {power})')

Part 1 answer - max power is at coordinates 243,16 (power was 31)


# Part 2

In [7]:
def part2(sn, convolve=fftconvolve):
    plGrid = powerLevelGrid(sn)
    max_power = -5
    x = None
    y = None
    best_size = None
    for size in range(1, 301):
        mpGrid = maxPowerGrid(plGrid, size, convolve=convolve)
        xi, yi, power = maxPowerFromGrid(mpGrid)
        if power > max_power:
            max_power = power
            x = xi
            y = yi
            best_size = size
    return x, y, best_size, max_power
    

In [8]:
part2_tests = [[18, 90, 269, 16, 113],
               [42, 232, 251, 12, 119]]

In [9]:
class Test2(unittest.TestCase):
    
    def test_p2(self):
        for sn, x, y, size, power in part2_tests:
            with self.subTest(sn=sn):
                self.assertEqual(part2(sn), (x, y, size, power))

# Run part 2 tests

In [10]:
suite = unittest.TestLoader().loadTestsFromTestCase(Test2)
unittest.TextTestRunner(verbosity=2).run(suite)

test_p2 (__main__.Test2) ... ok

----------------------------------------------------------------------
Ran 1 test in 9.462s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# Part 2 answer

In [11]:
%%timeit -n 1 -r 1
x, y, size, power = part2(day11_sn)
print(f'Part 2 answer -  max power is at coordinates {x},{y},{size} (power was {power})')

Part 2 answer -  max power is at coordinates 231,227,14 (power was 129)
4.65 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Performance comparison

SciPy has two 2d convolution routines `convolve2d` and `fftconvolve`.  `fftconvolve` (used above) is far quicker. For comparison, here is part 2 recalculated using `convolve2d`

See also: http://blog.invibe.net/posts/2017-09-20-the-fastest-2d-convolution-in-the-world.html

In [12]:
%%timeit -n 1 -r 1
x, y, size, power = part2(day11_sn, convolve=convolve2d)
print(f'Part 2 answer -  max power is at coordinates {x},{y},{size} (power was {power})')

Part 2 answer -  max power is at coordinates 231,227,14 (power was 129)
2min 47s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%prun part2(day11_sn, convolve=convolve2d)

         6349 function calls in 166.474 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      300  166.405    0.555  166.405    0.555 {built-in method scipy.signal.sigtools._convolve2d}
      300    0.024    0.000    0.024    0.000 {built-in method numpy.core.multiarray.where}
      300    0.010    0.000    0.047    0.000 <ipython-input-4-01e5c9a89e77>:25(maxPowerFromGrid)
      300    0.010    0.000    0.010    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      300    0.007    0.000    0.007    0.000 {method 'astype' of 'numpy.ndarray' objects}
      300    0.003    0.000  166.422    0.555 <ipython-input-4-01e5c9a89e77>:30(maxPowerGrid)
      300    0.002    0.000  166.409    0.555 signaltools.py:947(convolve2d)
      300    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.copyto}
      300    0.002    0.000    0.013    0.000 fromnumeric.py:2222(amax)
        1    0.002    0.002    0.003    0.003